In [1]:
from src.td_ml_probabilistic_unification.data_loader import *

/Users/gurbaksh.sharma/Desktop/td_code/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import pytd
# client = pytd.Client(database='test_dilyan')

# or, hard-code your API key, endpoint, and/or query engine:
client=pytd.Client(apikey=key, 
            endpoint='https://api.treasuredata.com', 
            database='test_dilyan', 
            default_engine='presto')

In [3]:
data=client.query("select * from prob_final_blocking_table_user_master", engine='presto')
# 

In [4]:
df = pd.DataFrame(data['data'], columns=data['columns'])

In [5]:
dup_data=df

In [6]:
import json
import os
import sys
import pandas as pd
import numpy as np
import datetime
# from .data_loader import TDConnector
import uuid

# os.system(f"{sys.executable} -m pip install fancyimpute==0.7.0")
# os.system(f"{sys.executable} -m pip install strsim==0.0.3")
# os.system(f"{sys.executable} -m pip install networkx==2.5")
# os.system(f"{sys.executable} -m pip install scipy==1.6.2")
# #os.system(f"{sys.executable} -m pip install pandiet==0.1.6")
# os.system(f"{sys.executable} -m pip install fuzzywuzzy==0.18.0")
# #os.system(f"{sys.executable} -m pip install memory_profiler==0.60.0")
# os.system(f"{sys.executable} -m pip install python-Levenshtein==0.21.1")


# #####----Defining Variables-------#####
# TD_SINK_DATABASE=os.environ.get('TD_SINK_DATABASE')
# TD_API_KEY=os.environ.get('TD_API_KEY')
# TD_API_SERVER=os.environ.get('TD_API_SERVER')

# id_col=os.environ.get('id_col')
# cluster_col_name=os.environ.get('cluster_col_name')
# convergence_threshold=float(os.environ.get('convergence_threshold'))
# cluster_threshold=float(os.environ.get('cluster_threshold'))
# string_type=os.environ.get('string_type')
# fill_missing=os.environ.get('fill_missing')
# feature_dict=json.loads(os.environ.get('feature_dict'))
# blocking_table=os.environ.get('blocking_table')
# output_table=os.environ.get('output_table')

# record_limit=int(os.environ.get('record_limit'))
# lower_limit=int(os.environ.get('lower_limit'))
# upper_limit=int(os.environ.get('upper_limit'))
# range_index=os.environ.get('range_index')
# paralelism = os.environ.get('paralelism')

# input_table=blocking_table



In [7]:
import os
import json

# You can override these by setting os.environ beforehand or fall back to these values

id_col = os.environ.get('id_col', 'persistent_id')
cluster_col_name = os.environ.get('cluster_col_name', 'prob_id')
convergence_threshold = float(os.environ.get('convergence_threshold', 0.01))
cluster_threshold = float(os.environ.get('cluster_threshold', 0.65))
string_type = os.environ.get('string_type', 'jarowinkler')
fill_missing = os.environ.get('fill_missing', 'True').lower() == 'true'
blocking_table = os.environ.get('blocking_table', 'prob_final_blocking_table_user_master')
output_table = os.environ.get('output_table', 'prob_final_user_master')

record_limit = int(os.environ.get('record_limit', 100000))
lower_limit = int(os.environ.get('lower_limit', 0))
upper_limit = int(os.environ.get('upper_limit', 0))  # set appropriately
range_index = os.environ.get('range_index', '0')
paralelism = os.environ.get('paralelism', '1')

# Handle feature_dict (this should be passed in as a JSON string in the environment)
feature_dict_json = os.environ.get('feature_dict', '''
[
    {
        "name": "phone_key",
        "type": "string",
        "weight": 0.35,
        "final_approval": "no"
    },
    {
        "name": "email_key",
        "type": "email",
        "weight": 0.35,
        "final_approval": "no"
    },
    {
        "name": "address_key",
        "type": "string",
        "weight": 0.30,
        "final_approval": "no"
    }
]
''')
feature_dict = json.loads(feature_dict_json)



feature_cols="block_key, " + id_col
for feature in feature_dict:
    name=feature['name']
    feature_cols=feature_cols +  ","  + name

In [8]:
from src.td_ml_probabilistic_unification.get_similarity import *
from src.td_ml_probabilistic_unification.get_cluster import *

In [9]:
import os

In [10]:
# New parameters
clustering_method = os.environ.get('clustering_method', 'safe')
min_coverage = float(os.environ.get('min_coverage', '0.5'))
min_cluster_size = int(os.environ.get('min_cluster_size', '2'))

def validate_feature_weights(feature_dict):
    """
    Validate and normalize feature weights to ensure they sum to 1.0.
    
    Args:
        feature_dict: List of feature dictionaries
    
    Returns:
        Validated feature dictionary with normalized weights
    """
    total_weight = sum(float(feature['weight']) for feature in feature_dict)
    
    if abs(total_weight - 1.0) > 0.01:  # Allow small tolerance
        print(f"Warning: Feature weights sum to {total_weight}, normalizing to 1.0")
        
        # Normalize weights
        for feature in feature_dict:
            feature['weight'] = float(feature['weight']) / total_weight
    
    return feature_dict


def apply_quality_filters(df_clusters, sim_data, id_col, cluster_col_name, 
                         min_cluster_size=2, min_avg_similarity=None):
    """
    Apply quality filters to remove low-quality clusters.
    
    Args:
        df_clusters: DataFrame with cluster assignments
        sim_data: Original similarity data
        id_col: ID column name
        cluster_col_name: Cluster column name
        min_cluster_size: Minimum number of records in cluster
        min_avg_similarity: Minimum average similarity in cluster
    
    Returns:
        Filtered DataFrame
    """
    if min_avg_similarity is None:
        min_avg_similarity = cluster_threshold
    
    # Count cluster sizes
    cluster_sizes = df_clusters[cluster_col_name].value_counts()
    valid_clusters = cluster_sizes[cluster_sizes >= min_cluster_size].index
    
    # Filter by cluster size
    df_filtered = df_clusters[df_clusters[cluster_col_name].isin(valid_clusters)].copy()
    
    # Filter by average similarity if specified
    if min_avg_similarity > 0:
        df_filtered = df_filtered[
            df_filtered['avg_cluster_similarity'] >= min_avg_similarity
        ].copy()
    
    # Add quality metrics
    df_filtered['cluster_size'] = df_filtered[cluster_col_name].map(cluster_sizes)
    
    return df_filtered

In [11]:
# Validate feature configuration
normalized_feature_dict = validate_feature_weights(feature_dict)

In [12]:
normalized_feature_dict

[{'name': 'phone_key',
  'type': 'string',
  'weight': 0.35,
  'final_approval': 'no'},
 {'name': 'email_key',
  'type': 'email',
  'weight': 0.35,
  'final_approval': 'no'},
 {'name': 'address_key',
  'type': 'string',
  'weight': 0.3,
  'final_approval': 'no'}]

In [13]:
import logging
import sys
import pandas as pd
# Simple logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
sim_data=pd.merge(dup_data,dup_data,on='block_key',suffixes=('_1','_2')).drop_duplicates()

#--- Dropping one of duplicate pairs e.g id_1=A and id_2= B ==> there will be two combinations A-B and B-A but we only need any one of them. so dropping one of them here
sim_data=sim_data[sim_data[id_col+'_1']>sim_data[id_col+'_2']]

# Calculate similarities with improved missing value handling
sim_data, sim_feat_list, col_names, weights = get_similarities(
    sim_data, 
    normalized_feature_dict, 
    string_type, 
    min_coverage=min_coverage
)

print(f"Generated {len(sim_data)} record pairs")
print(f"Pairs above threshold ({cluster_threshold}): {len(sim_data[sim_data['score'] >= cluster_threshold])}")
print(f"Average feature coverage: {sim_data['feature_coverage'].mean():.2f}")

2025-06-24 18:30:03,793 - INFO - Calculating similarities for 3654 pairs
2025-06-24 18:30:04,549 - INFO - Similarity calculation completed - mean score: 0.966
Generated 3654 record pairs
Pairs above threshold (0.65): 3650
Average feature coverage: 0.73


In [14]:
# Filter pairs that don't meet minimum requirements
valid_pairs = sim_data[
    (sim_data['score'] >= cluster_threshold) & 
    (sim_data['feature_coverage'] >= min_coverage)
]

print(f"Valid pairs after filtering: {len(valid_pairs)}")

if len(valid_pairs) == 0:
    print("No valid pairs found. Adjusting parameters...")
    # Relax coverage requirement
    relaxed_coverage = min_coverage * 0.7
    valid_pairs = sim_data[
        (sim_data['score'] >= cluster_threshold * 0.9) & 
        (sim_data['feature_coverage'] >= relaxed_coverage)
    ]
    print(f"Valid pairs with relaxed criteria: {len(valid_pairs)}")

# Perform clustering with improved method
df_clusters = clusters(
    sim_data, 
    id_col, 
    cluster_col_name, 
    cluster_threshold, 
    convergence_threshold, 
    col_names, 
    fill_missing, 
    clustering_method=clustering_method,
    min_coverage=min_coverage
)

print(f"Generated {df_clusters[cluster_col_name].nunique()} clusters")
print(f"Records in clusters: {len(df_clusters)}")

# Apply quality filters
df_filtered = apply_quality_filters(
    df_clusters, 
    sim_data, 
    id_col, 
    cluster_col_name, 
    min_cluster_size=min_cluster_size,
    min_avg_similarity=cluster_threshold
)

print(f"Clusters after quality filtering: {df_filtered[cluster_col_name].nunique()}")
print(f"Records after quality filtering: {len(df_filtered)}")

# Only keep clusters with multiple records (remove singletons)
cluster_counts = df_filtered[cluster_col_name].value_counts()
multi_record_clusters = cluster_counts[cluster_counts >= min_cluster_size].index
final_df = df_filtered[df_filtered[cluster_col_name].isin(multi_record_clusters)].copy()

if len(final_df) == 0:
    print("No clusters found after filtering. Creating empty result.")
    # Create empty DataFrame with correct structure
    final_df = pd.DataFrame(columns=[id_col, cluster_col_name, 'avg_cluster_similarity', 'cluster_size'])
    for col in col_names:
        final_df[col] = []
else:
    # Replace cluster ids with UUIDs
    unique_cluster_ids = final_df[cluster_col_name].unique()
    cluster_uuid_mapping = {cluster_id: str(uuid.uuid4()) for cluster_id in unique_cluster_ids}
    final_df[cluster_col_name] = final_df[cluster_col_name].map(cluster_uuid_mapping)

    # Merge with original data
    final_df = final_df.merge(dup_data, how='left', on=[id_col]).drop('block_key', axis=1).drop_duplicates()

    # Add range index prefix to cluster IDs
    final_df[cluster_col_name] = str(range_index) + '_' + final_df[cluster_col_name].astype('str')

    # Add quality metrics to output
    print(f"Final output statistics:")
    print(f"- Number of clusters: {final_df[cluster_col_name].nunique()}")
    print(f"- Number of records: {len(final_df)}")
    print(f"- Average cluster size: {final_df['cluster_size'].mean():.1f}")
    print(f"- Average cluster similarity: {final_df['avg_cluster_similarity'].mean():.3f}")
    print(f"- Minimum cluster similarity: {final_df['avg_cluster_similarity'].min():.3f}")


Valid pairs after filtering: 3650
2025-06-24 18:30:05,010 - INFO - Starting clustering with method 'safe', threshold 0.65
2025-06-24 18:30:05,149 - INFO - Clustering successful with method: safe
2025-06-24 18:30:09,358 - INFO - Clustering completed: 3432 clusters, 6927 records
Generated 3432 clusters
Records in clusters: 6927
Clusters after quality filtering: 3432
Records after quality filtering: 6927
Final output statistics:
- Number of clusters: 3432
- Number of records: 7058
- Average cluster size: 2.1
- Average cluster similarity: 0.967
- Minimum cluster similarity: 0.742


In [15]:
final_df

,prob_id,persistent_id,avg_cluster_similarity,cluster_size,email_key,phone_key,address_key,rnk,time
0,0_ca3f5908-b4b3-480b-858c-7e1bb13802a3,5ntDTml1Am7x,0.970000,2,VIRGINIA-R|-TREVINO-|VTREVINO279@GMAIL.COM,VIRGINIA-R|-TREVINO-|6237643404,None,4751,1749656326
1,0_ca3f5908-b4b3-480b-858c-7e1bb13802a3,L7HbkTiOKTXx,0.970000,2,VIRGINIA-R|TREVINO|VTREVINO279@GMAIL.COM,VIRGINIA-R|TREVINO|6237643404,None,4751,1749656326
2,0_98c19a54-dcbd-4422-93e4-02c7507db1be,zWvszRvZZuDx,0.977041,2,ARTHUR|RIDOLFI|ART.RIDOLFI@GMAIL.COM,ARTHUR|RIDOLFI|2032615581,None,1356,1749656326
3,0_98c19a54-dcbd-4422-93e4-02c7507db1be,4x1gCiNz-DTx,0.977041,2,ARTHUR|RIDOLFIL|ART.RIDOLFI@GMAIL.COM,ARTHUR|RIDOLFIL|2032615581,None,1356,1749656326
4,0_bf766d94-d513-451d-80ca-5f4e3fe60fd2,ZtCGEsXusu_x,0.999244,2,JERI|MOORE|MOOREJERI@GMAIL.CO,JERI|MOORE|5599449651,None,2959,1749656326
...,...,...,...,...,...,...,...,...,...
7053,0_e77c2370-c0ee-4564-9fa5-ceb7fede8d7b,I0Tyeqt3CyTx,0.985338,2,CHERYL|GRANT|GRAMMIEMATEO@GMAIL.COM,CHERYL|GRANT|8055268693,CHERYL|GRANT|1237 HAROLD AVE|93065,1512,1749656326
7054,0_e77c049d-8d61-4e82-8686-9e0b4a67c4d7,TGrNHaXdnOrx,0.980000,2,NASEEM|DOUCHAND|DOUCHAND.51@GMAIL.COM,NASEEM|DOUCHAND|3058787671,None,2813,1749656326
7055,0_e77c049d-8d61-4e82-8686-9e0b4a67c4d7,3P60h5BovRbx,0.980000,2,NASEEM|DOUCHAND-|DOUCHAND.51@GMAIL.COM,NASEEM|DOUCHAND-|3058787671,None,2813,1749656326
7056,0_9b1abe36-cde0-439b-a9da-05109cfabb66,OCVRFWXq2Cjx,0.964029,2,TINA|GARTLAND|TINAG803@GMAIL.COM,TINA|GARTLAND|4104994168,TINA|GARTLAND|803 S SHARP ST|21230,2999,1749656326
